# HabitLedger Demo – Behavioural Money Coach

Welcome to the **HabitLedger** interactive demonstration notebook!

## What is HabitLedger?

HabitLedger is an AI-powered behavioural money coach designed to help users build healthier financial habits through:

- **Behavioural Science Principles**: Uses proven concepts like loss aversion, habit loops, friction reduction, and commitment devices
- **Personalized Interventions**: Suggests specific, actionable strategies tailored to your situation
- **Memory & Tracking**: Remembers your goals, streaks, and struggles across interactions
- **Adaptive Coaching**: Detects patterns and adjusts recommendations over time

## What This Notebook Demonstrates

This notebook showcases:

1. How the agent analyzes user input using keyword-based principle detection
2. How it generates personalized coaching responses with interventions
3. How memory tracks progress across multiple interactions
4. Sample scenarios covering common financial habit challenges

**Note**: This demo uses deterministic keyword matching (no LLM calls yet) to demonstrate the core agent architecture and behaviour analysis logic.

## How This Notebook Is Organized

1. **Setup** — Import modules, load behaviour database, initialize user memory
2. **Single Interaction Demo** — See how the agent responds to one user input
3. **Multiple Interaction Scenarios** — Run through 3-5 predefined scenarios covering different behavioural principles
4. **Visuals & Structured Summaries** — Placeholder for future visualizations (e.g., streak charts, principle frequency)
5. **Evaluation Notes** — How to assess the agent's performance and relevance

## 1. Setup

Import modules, load the behaviour database, and initialize user memory.

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.memory import UserMemory
from src.behaviour_engine import analyse_behaviour, load_behaviour_db
from src.coach import run_once

# Load behaviour principles database
behaviour_db = load_behaviour_db("../data/behaviour_principles.json")
print(f"✅ Loaded {len(behaviour_db)} behavioural principles")

# Initialize user memory
user_memory = UserMemory(user_id="demo_user")
user_memory.goals = ["Save $500/month", "Stop impulse buying"]
print(f"✅ Initialized memory for user: {user_memory.user_id}")
print(f"   Goals: {user_memory.goals}")

## 2. Single Interaction Demo

Let's see how the agent responds to a single user input. The agent will:
1. Analyze the input for behavioural patterns
2. Detect the most relevant principle
3. Suggest personalized interventions
4. Update the user's memory

In [ ]:
# Example user input
user_input = "I keep buying coffee every morning even though I want to save money. It's become such a routine."

# Run the agent
response = run_once(
    user_input=user_input, memory=user_memory, behaviour_db=behaviour_db
)

print("🤖 Agent Response:")
print(response)

## 3. Multiple Interaction Scenarios

Now let's test the agent with several predefined scenarios that trigger different behavioural principles. This demonstrates how the agent adapts to different situations and tracks progress over time.

In [ ]:
# Define multiple test scenarios
scenarios = [
    {
        "name": "Loss Aversion",
        "input": "I'm afraid to check my bank account because I might see I've overspent again. It makes me anxious.",
    },
    {
        "name": "Temptation Bundling",
        "input": "I hate budgeting, it feels like such a chore. I keep putting it off.",
    },
    {
        "name": "Friction Reduction (Negative)",
        "input": "Online shopping is too easy. I just click and buy without thinking. No friction at all.",
    },
    {
        "name": "Micro Habits",
        "input": "Saving $500 a month feels overwhelming. I don't know where to start.",
    },
    {
        "name": "Default Effect",
        "input": "I'm still subscribed to services I never use. I just haven't bothered to cancel them.",
    },
]

# Run through each scenario
print("=" * 60)
for i, scenario in enumerate(scenarios, 1):
    print(f"\n📌 SCENARIO {i}: {scenario['name']}")
    print(f"User: {scenario['input']}")
    print("\n" + "-" * 60)

    response = run_once(
        user_input=scenario["input"], memory=user_memory, behaviour_db=behaviour_db
    )

    print(response)
    print("=" * 60)

## 4. Visuals and Structured Summaries (Placeholder)

This section is a placeholder for future visualizations and analytics:

- **Streak Charts**: Visualize consistency over time
- **Principle Frequency**: Show which behavioural patterns are most common
- **Progress Dashboard**: Track goal completion and intervention effectiveness
- **Behaviour Heatmaps**: Identify patterns by day/time

For now, we can inspect the memory state directly:

In [ ]:
# Inspect current memory state
import json

memory_state = user_memory.to_dict()
print("📊 Current User Memory State:")
print(json.dumps(memory_state, indent=2, default=str))

## 5. Evaluation Notes

### How to Evaluate HabitLedger

When assessing the agent's performance, consider:

#### 1. **Principle Detection Accuracy**
- Does the agent correctly identify the underlying behavioural principle?
- Are the detected triggers relevant to the user's input?

#### 2. **Intervention Relevance**
- Are the suggested interventions actionable and specific?
- Do they align with the detected principle and user's goals?

#### 3. **Memory & Consistency**
- Does the agent remember goals, streaks, and struggles across interactions?
- Does it reference past context appropriately?

#### 4. **Response Quality**
- Is the tone supportive and non-judgmental?
- Are explanations clear and grounded in behavioural science?

#### 5. **Multi-Step Behavior**
- Does the agent show adaptive behavior over multiple interactions?
- Does it recognize patterns and adjust recommendations?

### Current Limitations

- **Keyword-based detection**: Not yet using LLM for nuanced pattern recognition
- **No real-time data**: Memory persists within session but doesn't integrate with actual financial data
- **Limited context**: Doesn't yet maintain conversation history beyond memory fields

### Future Improvements

- Integrate LLM-based principle detection for better accuracy
- Add visualizations for streak tracking and progress monitoring
- Implement session summaries and weekly reviews
- Connect to financial APIs for real-time habit tracking